### Application of GPcounts with Negative Binomial likelihood on spatial data from Mouse olfactory bulb to identify spatially expressed genes. 

We first calculate the scale factors using negative binomial regression with identity link function using R glm package. 


For the following blocks switch to the R Kernel 

In [1]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("sva")
#library(sva)
library(MASS)

Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.1 (2019-07-05)
Installing package(s) 'BiocVersion', 'sva'
also installing the dependencies ‘ellipsis’, ‘vctrs’, ‘bit64’, ‘blob’, ‘IRanges’, ‘RSQLite’, ‘S4Vectors’, ‘RCurl’, ‘AnnotationDbi’, ‘annotate’, ‘Biobase’, ‘futile.logger’, ‘genefilter’, ‘BiocParallel’, ‘matrixStats’, ‘limma’

Warning message in install.packages(...):
“installation of package ‘matrixStats’ had non-zero exit status”Warning message in install.packages(...):
“installation of package ‘sva’ had non-zero exit status”Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Old packages: 'askpass', 'backports', 'BH', 'boot', 'broom', 'callr', 'caret',
  'class', 'cli', 'clipr', 'cluster', 'curl', 'data.table', 'dbplyr', 'digest',
  'dplyr', 'evaluate', 'fansi', 'forcats', 'foreach', 'formatR', 'fs',
  'ggplot2', 'glmnet', 'glue', 'gower', 'haven', 'hexbin', 'hms', 'htmltools',
  'htmlwidgets', 'httpuv', 'httr', 'ipred', 'IRkernel', 'iterators',
 

In [2]:
counts <- read.table(file='../data/MouseOB/Rep11_MOB_0.csv', header=TRUE, sep=',')
cells <- read.table(file='../data/MouseOB/locations.csv', header=TRUE, sep=',')

In [3]:
rownames(counts) = counts[,1]
rownames(cells) = cells[,1]
counts = counts[,-1]
cells = cells[,-1]

In [4]:
counts = cbind(counts,cells$total_counts)
colnames(counts)[14860] = 'total_counts'
coeffs_nb <-list()
scales_nb <- list()
total_counts = c(cells$total_counts)


In [5]:
for (x in c(1:100))
{
  results<-glm.nb(formula=counts[,x]~0+counts$total_counts,link=identity, data=counts)
  coeffs = as.data.frame(results$coefficients)
  scales <- results$coefficients*total_counts
  scales = as.data.frame(scales)
  scales_nb <- append(scales_nb,scales)
}

scales_nb=write.table(scales_nb,sep = "\t","../data/MouseOB/scales_nb.txt",col.names = TRUE, row.names = F)


Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”

Switch to Python kernel

In [6]:
import pandas as pd 
import numpy as np 
import gpflow
import tensorflow as tf
from GPcounts import NegativeBinomialLikelihood
from GPcounts.GPcounts_Module import Fit_GPcounts
from matplotlib import pyplot as plt

ERROR: Error in parse(text = x, srcfile = src): <text>:1:8: unexpected symbol
1: import pandas
           ^


In [ ]:
def get_coords(index):
    coords = pd.DataFrame(index=index)
    coords['x'] = index.str.split('x').str.get(0).map(float)
    coords['y'] = index.str.split('x').str.get(1).map(float)
    return coords

In [ ]:
# spatial expression counts matrix
Y = pd.read_csv('../data/MouseOB/Rep11_MOB_0.csv', index_col=[0])
Y = Y.T[Y.sum(0) >= 3].T

In [ ]:
sample_info = get_coords(Y.index)
sample_info['total_counts'] = Y.sum(1)
sample_info = sample_info.query('total_counts > 10')  # Remove empty features
Y = Y.loc[sample_info.index]
X = sample_info[['x', 'y']]
scale=pd.read_csv('../data/MouseOB/scales_nb.txt', sep="\t")

In [ ]:
Y=Y.T
Y=Y.iloc[0:100,:] # select first 100 genes to run GPcounts
X = X[['x', 'y']]

In [ ]:
gene_name = []
scaled = True # set the default scaled argument to True to consider the scale factors
gene_name = Y.index
likelihood = 'Negative_binomial' 
gp_counts = Fit_GPcounts(X,Y.loc[gene_name],scale.loc[scale.index], scaled = scaled)

In [ ]:
log_likelihood_ratio = gp_counts.One_sample_test(likelihood)
print(log_likelihood_ratio)

Plot some spatially expressed genes from the data set

In [ ]:
df = log_likelihood_ratio.sort_values(by=['q value'])
df.head(20)

In [ ]:
from matplotlib import rcParams
plt.rcParams["figure.figsize"] = [10,3]
m = ['Pcp4', 'Fyco1', 'Eef1a1']
for i, g in enumerate(['Pcp4', 'Fyco1', 'Eef1a1']):
    plt.subplot(1, 3, i + 1)
    plt.scatter(sample_info['x'], sample_info['y'], c=Y.T[g],cmap = 'summer');
    # plt.figure(figsize=(10,3))
    plt.title(m[i])
    plt.axis('equal')


plt.colorbar(ticks=[0,1],label = 'Gene Epxression');

plt.show()
